In [6]:
import code

import numpy as np
import gurobipy as gp

model = code.model.Model(n = 1, k_half = 3, k_upper = 9)

model.set_objective([
    (1, [0]),
    (1, [1]),
    (-3, [3]),
    (2, [4]),
])

model.add_cone(
    'bound', 
    localizer = [
        (0.1, [0]),
        (1, [1])
    ],
    inequality = True,
)

model.optimize(60)

model.vars_x

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-6700HQ CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 33 rows, 160 columns and 2041 nonzeros
Model fingerprint: 0x1e2b1677
Coefficient statistics:
  Matrix range     [3e-07, 1e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 14 rows and 12 columns
Presolve time: 0.01s
Presolved: 19 rows, 148 columns, 1214 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
      91    9.0326885e-01   0.000000e+00   0.000000e+00      0s

Solved in 91 iterations and 0.01 seconds (0.00 work units)
Optimal objective  9.032688479e-01


{0: <gurobi.Var x[0] (value 1.0)>,
 1: <gurobi.Var x[1] (value -0.09960092292665351)>,
 2: <gurobi.Var x[2] (value 0.010361269827099104)>,
 3: <gurobi.Var x[3] (value -0.0006227306555921172)>,
 4: <gurobi.Var x[4] (value 0.0005007894370221393)>,
 5: <gurobi.Var x[5] (value 0.0005056935259183911)>,
 6: <gurobi.Var x[6] (value 0.0019335898480355702)>,
 7: <gurobi.Var x[7] (value 0.02415782947765833)>,
 8: <gurobi.Var x[8] (value 0.0)>,
 9: <gurobi.Var x[9] (value 0.0)>}

In [ ]:
# Check the status of the solution
if model.status == gp.GRB.Status.INFEASIBLE:
    print('The model is infeasible')

    # Compute the IIS
    model.computeIIS()

    # Print the IIS
    for c in model.getConstrs():
        if c.IISConstr:
            print(f'Constraint {c.ConstrName} is part of the IIS')

    # Retrieve and print the dual extreme ray (Farkas dual)
    for c in model.getConstrs():
        print(f'Dual Farkas value for constraint {c.ConstrName}: {c.dualFarkas}')
else:
    print('The model has a feasible solution')

The model has a feasible solution
